# Lab 12: Probability and Distance

This lab is presented with some revisions from [Dennis Sun at Cal Poly](https://web.calpoly.edu/~dsun09/index.html) and his [Data301 Course](http://users.csc.calpoly.edu/~dsun09/data301/lectures.html)

### When you have filled out all the questions, submit via [Tulane Canvas](https://tulane.instructure.com/)

# Part 1: Probability
So far, we have seen different ways to summarize and visualize _individual_ variables in a data set. But we have not really discussed how to summarize and visualize relationships between _multiple_ variables. This chapter is all about how to understand relationships between the columns in a `DataFrame`. The methods will be different, depending on whether the variables are categorical or quantitative.

In [ ]:
# first, mount your google drive, change to the course folder, pull latest changes, and change to the lab folder.
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/cmps3160
!git pull
%cd _labs/Lab12

In this section, we look at ways to summarize the relationship between two _categorical_ variables. To do this, we will again use the Titanic data set.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

titanic_df = pd.read_csv("../data/titanic.csv")

Suppose we want to understand the relationship between where a passenger embarked and what class they were in. We can completely summarize this relationship by counting the number of passengers in each class that embarked at each location. We can create a pivot table that summarizes this information.

In [ ]:
embarked_pclass_counts = titanic_df.pivot_table(
    index="embarked", columns="pclass",
    values="name",  # We can pretty much count any column, as long as there are no NaNs.
    aggfunc="count" # The count function will count the number of non-null values.
)
embarked_pclass_counts

Recall that the field embarked is coded categorically using: `embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)`

A pivot table that stores counts is also called a **contigency table** or a **cross-tabulation**. This type of pivot table is common enough that there is a specific function in `pandas` to calculate one, allowing you to bypass `.pivot_table`:

In [ ]:
counts = pd.crosstab(titanic_df.embarked, titanic_df.pclass)
counts

## Joint Distributions

It is common to normalize the counts in a table so that they add up to 1. These proportions represent the **joint distribution** of the two variables.

To calculate the joint distribution, we need to divide the table of counts above by the total count. To find the total count, we call `.sum()` twice; the first call gives us the sum of each column, and the second call adds those numbers together.

In [ ]:
print(counts.sum().sum())
joint = counts / counts.sum().sum()
joint

Note that this is yet another example of broadcasting. When we divided the `DataFrame` `counts` by the number 1307, the division was applied elementwise, producing another `DataFrame`.

Each cell in this `DataFrame` tells us a joint proportion. For example, the cell in the bottom right tells us the proportion of all passengers that embarked at Southampton and were in 3rd class. We notate this joint proportion as follows:

$$ P(\text{embarked at Southampton and in 3rd class}) = .379. $$

The joint distribution above could also have been obtained by specifying `normalize=True` when the contingency table was first created:

In [ ]:
pd.crosstab(titanic_df.embarked, titanic_df.pclass,
            normalize=True)

The above joint distribution is not, strictly speaking, a contingency table. A contingency table is a table of all counts, while the above table is a table of proportions.

## Marginal Distributions

The **marginal distribution** of a variable is simply the distribution of that variable, ignoring the other variables. To calculate the marginal distribution from a joint distribution of two variables, we sum the rows or the columns of the joint distribution.

For example, to calculate the marginal distribution of `embarked`, we have to sum the joint distribution over the columns---in other words, _roll-up_ or _marginalize over_ the `pclass` variable:

In [ ]:
joint.sum(axis=1)

We can check this answer by calculating the distribution of `embarked` directly from the original data, ignoring `pclass` entirely.

In [ ]:
embarked_counts = titanic_df.groupby("embarked")["name"].count()
embarked_counts / embarked_counts.sum()

The numbers match!

Likewise, we calculate the marginal distribution of `pclass` by summing the joint distribution over the rows---in other words, by _rolling-up_ or _marginalizing over_ the `embarked` variable:

In [ ]:
joint.sum(axis=0)

So given the joint distribution of two categorical variables, there are two marginal distributions: one for each of the variables. These marginal distributions are obtained by summing the joint distribution table over the rows and over the columns.

The _marginal distribution_ is so-named because these row and column totals would typically be included alongside the joint distribution, in the _margins_ of the table. A contingency table with the marginal distributions included can be obtained by specifying `margins=True` in `pd.crosstab`:

In [ ]:
pd.crosstab(titanic_df.embarked, titanic_df.pclass,
            normalize=True, margins=True)

Notice in the above that the sum over all elements of the join distribution (the bottom right corner) is a probability distribution (sums to 1.0).  Likewise the sum over either of the computed marginal distributions is also a probability distribution (sums to 1.0).

## Conditional Distributions

The **conditional distribution** tells us about the distribution of one variable, _conditional on_ the value of another. For example, we might want to know the proportion of 3rd class passengers that embarked at each location. In other words, what is the distribution of where a passenger embarked, _conditional on_ being in 3rd class?

If we go back to the contingency table:

In [ ]:
embarked_pclass_counts

there were $101 + 113 + 495 = 709$ passengers in 3rd class, of whom 

- $101 / 709 = .142$ were in 1st class,
- $113 / 709 = .159$ were in 2nd class, and
- $495 / 709 = .698$ were in 3rd class.

We can calculate these proportions in code by dividing the `pclass=3` column by its sum:

In [ ]:
embarked_pclass_counts[3] / embarked_pclass_counts[3].sum()

Notice that these three proportions add up to 1, making this a proper distribution.

This conditional distribution helps us answer questions such as, "What proportion of 3rd class passengers embarked at Southampton?" We notate this conditional proportion as follows:

$$ P\big(\textrm{embarked at Southampton}\ \big|\ \textrm{in 3rd class}\big) = 0.698. $$

The pipe $\big|$ is read "given". So we are interested in the proportion of passengers who embarked at Southampton, _given_ that they were in 3rd class.

We could have also calculated this conditional distribution from the joint distribution (i.e., proportions instead of counts):

In [ ]:
joint[3] / joint[3].sum()

We have just calculated _one_ of the conditional distributions of `embarked`: the distribution conditional on being in 3rd class. There are two more conditional distributions of `embarked`: 

- the distribution conditional on being in 1st class 
- the distribution conditional on being in 2nd class

It is common to report _all_ of the conditional distributions of one variable given another variable.

Of course, it is straightforward to calculate these conditional distributions manually:

In [ ]:
embarked_pclass_counts[1] / embarked_pclass_counts[1].sum()

In [ ]:
embarked_pclass_counts[2] / embarked_pclass_counts[2].sum()

But there is a nifty trick for calculating all three conditional distributions at once. By summing the counts over `embarked`, we obtain the total number of people in each `pclass`:

In [ ]:
pclass_counts = embarked_pclass_counts.sum(axis=0)
pclass_counts

This is exactly what we need to divide each column of `embarked_pclass_counts` by:

In [ ]:
embarked_given_pclass = embarked_pclass_counts.divide(pclass_counts, axis=1)
embarked_given_pclass

(This is yet another example of _broadcasting_, since we are dividing a `DataFrame` by a `Series`.)

Compare each column with the numbers we obtained earlier. Notice also that each column sums to 1, a reminder that each column represents a separate distribution.

When comparing numbers across distributions, it is important to be careful. For example, the 87.4% and the 69.8% in the "Southampton" row represent **percentages of different populations that have different sizes!**. Just because 87.4% is higher than 69.8% does not mean that more 2nd class passengers boarded at Southampton than 3rd class passengers. In fact, if we go back to the original contingency table, we see that more 3rd class passengers actually boarded at Southampton than 2nd class passengers!

There is also another set of conditional distributions for these two variables: the distribution of class, conditional on where they embarked. To calculate these conditional distributions, we instead divide `embarked_pclass_counts` by the sum of each row:

In [ ]:
embarked_counts = embarked_pclass_counts.sum(axis=1)
pclass_given_embarked = embarked_pclass_counts.divide(embarked_counts, axis=0)
pclass_given_embarked

These conditional distributions answer questions like, "What proportion of Southampton passengers were in 3rd class?" 

Notice that these proportions are _not_ the same as the proportions for the other set of conditional distributions. That is because the two questions below are fundamentally different:

_Question 1._ What proportion of 3rd class passengers embarked at Southampton?

$$P\big(\textrm{embarked at Southampton}\ \big|\ \textrm{in 3rd class}\big) = \frac{\text{# passengers who embarked at Southampton and in 3rd class}}{\text{# passengers who in 3rd class}}$$

_Question 2._ What proportion of Southampton passengers were in 3rd class? 

$$P\big(\textrm{in 3rd class}\ \big|\ \textrm{embarked at Southampton}\big) = \frac{\text{# passengers who embarked at Southampton and in 3rd class}}{\text{# passengers who embarked at Southampton}} \\ $$



In the first case, the reference population is all passengers who embarked at Southampton. In the second case, the reference population is all passengers who were in 3rd class. The numerators may be the same, but the denominators are different. In general, the conditional distributions of $X$ given $Y$ are _not_ the same as the conditional distributions of $Y$ given $X$. 

If we rephrase the question slightly, we get yet another answer:

_Question 3._ What proportion of passengers embarked at Southampton _and_ were in 3rd class?

$$P(\text{embarked at Southampton and in 3rd class}) = \frac{\text{# passengers who embarked at Southampton and in 3rd class}}{\text{# passengers (total)}}$$

The reference population here is all passengers. This is the proportion that one would get from the joint distribution.

It is important to pay attention to the wording of the question, to determine whether a joint distribution or a conditional distribution is called for---and, if the latter, which of the two conditional distributions is appropriate.

## Visualization

How do we visualize the joint and conditional distributions of two categorical variables? (Marginal distributions are summaries of a single variable and can be visualized using the techniques of Chapter 1.)

To visualize a joint distribution, we need to be able to represent three dimensions: two dimensions for the two categorical variables and a third dimension for the proportions. Although one option is a 3D graph, humans are not good at judging the sizes of 3D objects printed on a page. For this reason, **heat maps**, which use a color scale to represent the third dimension, are usually preferred. 

Unfortunately, heat maps are still not easy to create in `pandas`. We use the `seaborn` library to make a heat map:

In [ ]:
import seaborn as sns

sns.heatmap(joint)

A heat map encourages comparison across cells. So we see that 3rd class passengers who embarked at Southampton were by far the most common.

Although a heat map can also be used to visualize conditional distributions, it is not ideal because it does not tell us which variable we are conditioning on, and it is difficult to judge visually which dimension sums to 1. A stacked bar graph is better because it visually shows values summing to 1.

To make a stacked bar graph, we simply specify `stacked=True` in `.plot.bar()`, to get the bars to show up on top of one another, instead of side-by-side:

In [ ]:
pclass_given_embarked.plot.bar(stacked=True)

However, the same code does not work on the other set of conditional distributions:

In [ ]:
embarked_given_pclass.plot.bar(stacked=True)

What went wrong? Recall that `.plot.bar()` automatically plots the (row) index of the `DataFrame` on the $x$-axis. To plot the distribution of `embarked` conditional on `pclass`, we need `pclass` to be on the $x$-axis, but 

In [ ]:
embarked_given_pclass

has `embarked` as the index. To make `pclass` the index, we can **transpose** this `DataFrame` so that the rows become columns and the columns become rows. The syntax for transposing a `DataFrame` is `.T`, which is inspired by the notation for transposing a matrix in linear algebra.

In [ ]:
embarked_given_pclass.T

Now, we can make a stacked bar graph from this _transposed_ `DataFrame`:

In [ ]:
(embarked_given_pclass.T).plot.bar(stacked=True)

## Summing Up About Conditionals Probabilities

We have discussed three different types of distributions (probabilities).  Remember that for two events $X$ and $Y$:

* Joint Distribution:  The probabilitiy of two events occuring at the same time.  Formally: $P(X \cap Y)$.  We typically use heatmaps to show joint distributions.
* Marginal Distribution: The probabilitiy of a single event, irrespective of any other event (i.e., marginalizing the other event).  Formally: $P(X)$.  We can use the graphs learned before (pie charts, bar graphs) to visualize these.
* Conditional Distribution:  The probability of an event $X$ happening given that some event $Y$ has also happened.  Formally: $P(X|Y) = \frac{P(X \cap Y)}{P(Y)}$.  We typically use stacked bar graphs to visualize these distributions.

When comparing across variables it is important to think about the relationship you want to present so that you can clearly and precisely discuss the relationship using the concepts from this notebook.

##  Independence

We would like to measure the strength of the relationship between two variables. _Independence_ is a way to quantify the intuitive notion that two variables are _unrelated_. Once we have defined independence, we can quantify the relationship between two variables by calculating how far they are from independence.

Formally, two variables $X$ and $Y$ are **independent** if the conditional distributions of $Y$ given $X$ (or vice versa) are all _identical_. In other words, the value of $X$ does not affect the distribution of $Y$.

In [ ]:
titanic_df["adult"] = (titanic_df["age"] >= 18)

For example, consider the relationship between sex and age group (adult or not). First, let's calculate the contingency table:

In [ ]:
counts = pd.crosstab(titanic_df.sex, titanic_df.adult)
counts

Although there are more male adults (576) than female adults (316), the _conditional proportion_ of adults, given sex, are actually very close (about $0.68$).

In [ ]:
adult_given_sex = counts.divide(counts.sum(axis=1), axis=0)
adult_given_sex.plot.bar(stacked=True)
adult_given_sex 

Because the conditional distribution of `adult` is (approximately) the same, regardless of whether we are conditioning on `sex` = male or `sex` = female, we say that the two variables are (approximately) independent. 

For an example of two non-independent variables, consider passenger class and age group. If we look at the conditional distributions of `adult` given `pclass`, they are not all the same:

In [ ]:
adult_pclass_counts = pd.crosstab(titanic_df.pclass, titanic_df.adult)
(adult_pclass_counts.divide(
    adult_pclass_counts.sum(axis=1), axis=0)
).plot.bar(stacked=True)

The conditional distribution of `adult` given `pclass` = 3 is quite different from the other two conditional distributions. Because the conditional distributions are not all equal, the two variables are _not_ independent. Note that it only takes _one_ conditional distribution to be off to render two variables _not_ independent.

## The Joint Distribution Assuming Independence

What would the joint distribution of passenger class (`pclass`) and age group (`adult`) be, if the two variables were independent? If two variables are independent, then their joint distribution is the product of the marginal distributions. That is,

- $P(\text{1st class and adult}) = P(\text{1st class}) \cdot P(\text{adult})$
- $P(\text{2nd class and adult}) = P(\text{2nd class}) \cdot P(\text{adult})$
- $P(\text{3rd class and adult}) = P(\text{3rd class}) \cdot P(\text{adult})$
- $P(\text{1st class and not adult}) = P(\text{1st class}) \cdot P(\text{not adult})$
- $P(\text{2nd class and not adult}) = P(\text{2nd class}) \cdot P(\text{not adult})$
- $P(\text{3rd class and not adult}) = P(\text{3rd class}) \cdot P(\text{not adult})$

We can calculate the marginal distributions:

In [ ]:
# Calculate the total number of passengers.
N = adult_pclass_counts.sum().sum()

# Calculate the marginal distribution of adult by summing over pclass.
adult = adult_pclass_counts.sum(axis=0) / N
adult

In [ ]:
# Calculate the marginal distributio of pclass by summing over adult.
pclass = adult_pclass_counts.sum(axis=1) / N
pclass

How do we multiply these two distributions to get a $3 \times 2$ table of the joint distribution, assuming independence? We can use matrix multiplication. We can think of one `Series` as a matrix with 1 column and the other as a matrix with 1 row. Multiplying the two matrices using the usual definition of matrix multiplication gives the desired joint proportions.

$$ {\bf u} {\bf v}^T = \begin{pmatrix} u_1 \\ u_2 \\ u_3 \end{pmatrix} \begin{pmatrix} v_1 & v_2 \end{pmatrix} = \begin{pmatrix} u_1 v_1 & u_1 v_2 \\ u_2 v_1 & u_2 v_2 \\ u_3 v_1 & u_3 v_2 \end{pmatrix} $$

This is an operation in linear algebra known as an **outer product**. To calculate the outer product of two `numpy` arrays, we can use the function `np.outer`:

In [ ]:
np.outer(pclass, adult)

Note that this returns a plain `numpy` `array` instead of a `pandas` `DataFrame`. It turns out that this will be good enough for our purposes.  You can always turn this into a DataFrame by passing it to `pd.DataFrame`.

## Measuring Distance from Independence

We now have, for every combination of our two variables, two proportions:

- the proportion that was actually observed, $P(A \text{ and } B)$
- the proportion that we would expect assuming independence, $P(A) P(B)$

To measure the relationship between two variables, we calculate how far the observed proportions are from what we would expect if the variables were independent. It turns out that there are several ways to calculate the "distance" between two distributions.

**Total Variation Distance**

**Total variation distance** is probably the first distance metric that comes to mind. We calculate the difference and take absolute values before summing so that negative errors don't cancel out positive ones (the motivation for taking absolute values is the same as in MAD, which we learned in Chapter 1): 

$$ TV = \sum_{\text{A, B}} \big|P(A \text{ and } B) - P(A) P(B)\big|. $$

In [ ]:
joint = adult_pclass_counts / N
expected = np.outer(pclass, adult)

# Total Variation Distance
(joint - expected).abs().sum().sum()

Unfortunately, differences turn out to be a bad way to measure distances between proportions. For example, most people would agree that the difference between $0.42$ and $0.41$ is insignificant, but the difference between $0.01$ and $0.00$ is vast. But total variation distance treats both differences the same.

**Chi-Square Distance**

**Chi-square distance** solves the problem of total variation distance by dividing by the difference by expected proportion, effectively calculating the _relative_ difference between the two proportions:

$$ \chi^2 = \sum_{\text{A, B}} \frac{(P(A \text{ and } B) - P(A) P(B))^2}{P(A) P(B)}. $$

In [ ]:
(((joint - expected) ** 2) / expected).sum().sum()

You might be familiar with the chi-square test from a previous statistics class. The chi-square distance is essentially the same as the chi-square test statistic, except for a normalizing constant.

**Mutual Information**

Another popular distance metric is **mutual information**. Whereas chi-square distance tends to be more popular among statisticians, mututal information tends to be more popular among engineers. (It arises from a field called _information theory_ which you can take a course on here at Tulane if you wish!.)

$$ I = \sum_{\text{A, B}} P(A \text{ and } B) \log \left( \frac{P(A \text{ and } B)}{P(A) P(B)} \right) $$

In [ ]:
(joint * np.log(joint / expected)).sum().sum()

There is no best distance metric for measuring departures from independence. All three distance metrics above are used in practice. The distances themselves can also be difficult to interpret. But the distance metric can give a rough sense of how closely two variables are related.

# Exercises, Part 1

Exercises 1-4 deal with the Tips data set (`../data/tips.csv`).  Load this dataset below.

In [ ]:
# CODE HERE
df_tips = pd.read_csv("../data/tips.csv")
df_tips.head()

**Exercise 1.** Make a visualization (both a table and a graph) that displays the relationship between the day of the week and party size.  Which type of distribution did you pick to represent this?  Why?

Hint: There may be some missing data that you need to make sure you are [handling correctly](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html).

In [ ]:
# ENTER YOUR CODE HERE

**Exercise 2.** Calculate the marginal distribution of day of week in two different ways using the results from the last cell.

In [ ]:
# ENTER YOUR CODE HERE

**Exercise 3.** Make a visualization and display the DataFrame that displays the conditional distribution of party size, given the day of the week.  That is, show $P(\text{party size} | \text{day of week})$.

In [ ]:
# ENTER YOUR CODE HERE

**Exercise 4.** What proportion of Saturday parties had 2 people? Is this the same as the proportion of 2-person parties that dined on Saturday?  Give the equation for both of these and explain the difference

In [ ]:
# ENTER YOUR CODE HERE

**Math Answer Here.**

**Exercise 5.** Report a measure of the strength of the relationship between the size of the party and the day of the week.  Display the count table and the conditional distribution table as well. 

In [ ]:
# YOUR CODE HERE 

**Bonus Bonus 5 Point Exercise.** We discussed above that the conditional distributions of A given B and the conditional distributions of B given A are _not_ the same. 

Can you figure out a way to relate the two? 

Can you write code that will convert a table with the conditional distributions of A given B, into a table with the conditional distributions of B given A?

To get complete credit you must write code and write the mathmatical formula you use (in math mode offset by \$ symbols) and name the formula correctly.



In [ ]:
# ENTER YOUR CODE HERE

**Math Answer Here.**

# Part 2: Distance Metrics

This lab is presented with some revisions from [Dennis Sun at Cal Poly](https://web.calpoly.edu/~dsun09/index.html) and his [Data301 Course](http://users.csc.calpoly.edu/~dsun09/data301/lectures.html)

### When you have filled out all the questions, submit via [Tulane Canvas](https://tulane.instructure.com/)

The previous labs we discussed ways to measure relationships between variables, or the _columns_ of a `DataFrame`. This chapter is about how to measure relationships between observations, or the _rows_ of a `DataFrame`. How do we quantify how "similar" two observations are? We will use the Ames housing data set, but to keep things simple, we will work with just three quantitative variables from that data set: the number of bedrooms, the number of bathrooms, and the living area (in square feet).

In [ ]:

housing_df = pd.read_csv("../data/ames.tsv", sep="\t")

# extract 3 quantitative variables
housing_df_quant = housing_df[["Bedroom AbvGr", "Gr Liv Area"]].copy()
housing_df_quant["Bathrooms"] = (
    housing_df["Full Bath"] + 
    0.5 * housing_df["Half Bath"]
)
housing_df_quant

Shown below is a (three-dimensional) scatterplot of these variables. Consider the two observations connected by a red line. (The label next to each point is its index in the `DataFrame`.) To measure how similar they are, we can calculate the distance between the two points.

<img src="../images/distance.png">

Calculating the distance between two points is not as straightforward as it might seem because there is more than one way to define distance. The one most familiar to you is probably **Euclidan distance**, which is the straight-line distance ("as the crow flies") between the two points. The formula for calculating this distance is a generalization of the Pythagorean theorem:

$$ d({\bf x}, {\bf x'}) = \sqrt{\sum_{j=1}^D (x_j - x'_j)^2} $$

Which we've seen before as the sum of squared distances!

In [ ]:
x = housing_df_quant.loc[2927]
x1 = housing_df_quant.loc[2928]

x - x1

In [ ]:
(x - x1) ** 2

In [ ]:
np.sqrt(((x - x1) ** 2).sum())

The beauty of this definition is that it generalizes to more than three dimensions. Even though it is difficult to visualize points in 100-dimensional space, we can calculate distances between them in exactly the same way.

However, Euclidean distance is not the only way to measure how far apart two points are. There is also [**Manhattan distance**](https://en.wikipedia.org/wiki/Taxicab_geometry) (also called _taxicab distance_ ), which measures the distance a taxicab in Manhattan would have to drive to travel from A to B. Taxicabs are not able to travel in a straight line (i.e., the green path below, the Euclidian distance) because they have to follow the street grid. But there are multiple paths along the street grid that all have exactly the same length (i.e., the red, yellow, and blue paths below); the Manhattan distance is the length of any one of these shortest paths.

<img src="../images/dist.png">

The formula for Manhattan distance is actually quite similar to the formula for Euclidean distance. Instead of squaring the differences and taking the square root at the end (as in Euclidean distance), we simply take absolute values:
$$ d({\bf x}, {\bf x'}) = \sum_{j=1}^D |x_j - x'_j|. $$

The following code calculates Manhattan distance:

In [ ]:
((x - x1).abs()).sum()

### Comparison of Euclidean and Manhattan distance

The Euclidean distance was essentially just the largest difference. This is because Euclidean distance first _squares_ the differences. The squaring operation has a "rich get richer" effect; larger values get magnified by more than smaller values. As a result, the largest differences tend to dominate the Euclidean distance.

On the other hand, Manhattan distance treats all differences equally. So Manhattan distance is preferred if you are concerned that an outlier in one variable might dominate the distance metric.

## The Importance of Scaling

Here's a quiz. There are two pairs of observations in the figure below, one connected by a red line, the other connected by an orange line. Which pair of observations is more similar (assuming we use Euclidean distance)?

![](../images/closer.png)

Let's actually calculate these two distances.

In [ ]:
# Distance between two points connected by red line
x = housing_df_quant.loc[2927]
x1 = housing_df_quant.loc[2928]

np.sqrt(((x - x1) ** 2).sum())

In [ ]:
# Distance between two points connected by orange line
x = housing_df_quant.loc[2498]
x1 = housing_df_quant.loc[290]

np.sqrt(((x - x1) ** 2).sum())

Surprised by the answer? The scatterplot is deceiving because it automatically scales the variables to make the points fit on the same plot. In reality, the variables are on very different scales. The number of bedrooms and bathrooms range from 0 to 6, while living area is in the thousands. When variables are on such different scales, the variable with the largest variability will dominate the distance metric.

The plot below shows the same data, but drawn to scale. You can see that differences in the number of bedrooms and the number of bathrooms hardly matter at all; only the variability in the living area matters.

![](../images/closer_rescaled.png)

To obtain distances that agree more with our intuition---and that do not give too much weight to any one variable---we transform the variables to be on the same scale. There are a few ways to **scale** a variable:

- **standardizing**: subtract each variable by its mean, then divide by its standard deviation, (also called z-standardization, 
$$ x_i \leftarrow \frac{x_i - \text{mean}[X]}{\text{SD}[X]} $$
- **normalizing**: scale each variable to have length (or "norm") 1, 
$$ x_i \leftarrow \frac{x_i}{\sqrt{\sum_{i=1}^n x_i^2}} $$
- **min/max scaling**: scale each variable so that all values are between 0 and 1, 
$$x_i \leftarrow \frac{x_i - \min[X]}{\max[X] - \min[X]}.$$

The figure below illustrates what each of these scaling methods do to a synthetic data set with two variables. All three methods scale the variables in similar (but slightly different) ways, resulting in figure-eights with different aspect ratios.  Standardizing also moves the data to be centered around the origin, while min-max scaling moves the data to be in a box whose corners are $(0, 0)$ and $(1, 1)$.

![](../images/scaling.png)

Let's standardize the Ames housing data, and see how it affects the distance metric.

In [ ]:
housing_df_std = (
    (housing_df_quant - housing_df_quant.mean()) / 
    housing_df_quant.std()
)
housing_df_std

Notice that the resulting `DataFrame` contains negative values. This makes sense because standardizing makes the mean of every variable equal to 0. If the mean is 0, then some values must be negative.

The above command is deceptively simple. We actually subtracted a `DataFrame` by a `Series`, then divided the resulting `DataFrame` by another `Series`. We relied on `pandas` to broadcast each `Series` over the right dimension of the `DataFrame`. To be more explicit about the broadcasting, we could have also used the `.sub()` and `.divide()` methods (instead of `-` and `/`) and been explicit about the axis:

In [ ]:
housing_df_std = (housing_df_quant.
                  sub(housing_df_quant.mean(), axis=1).
                  divide(housing_df_quant.std(), axis=1))
housing_df_std

Now let's recalculate the distances using this standardized data and see if our conclusions change.

In [ ]:
# Distance between two points connected by red line
x = housing_df_std.loc[2927]
x1 = housing_df_std.loc[2928]

np.sqrt(((x - x1) ** 2).sum())

In [ ]:
# Distance between two points connected by orange line
x = housing_df_std.loc[2498]
x1 = housing_df_std.loc[290]

np.sqrt(((x - x1) ** 2).sum())

So, if we first standardize the data, then the pair of observations connected by the red line are more similar than the pair connected by the orange line, which matches our intuition. It is (almost) always a good idea to scale your variables before calculating distances.

Now that you've seen how to implement one scaling method (standardization), you will implement two more (normalization and min-max scaling) in Exercises 1 and 2 below.

# Exercises, Part 2

**Exercise 1.** Instead of standardizing the three variables from the Ames housing data set, normalize them. Then, recompute the distances between the two pairs of points above. Does your conclusion change?

In [ ]:
# YOUR CODE HERE

**Written Answers Here:**

**Exercise 2.** Instead of standardizing the three variables from the Ames housing data set, apply a min-max scaling to them. Then, recompute the distances between the two pairs of points above. Does your conclusion change?

In [ ]:
# YOUR CODE HERE

**Written Answers Here:**

Exercises 3-5 ask you to work with a data set that describes the chemical composition of 1599 red wines (`../data/reds.csv`). There are 12 variables in this data set, all of which are quantitative (so each observation is a point in 12-dimensional space).

In [ ]:
df_reds = pd.read_csv("../data/reds.csv", sep=';')
df_reds[:5]

**Exercise 3.** Find which red wine is more similar to wine 0 in the `DataFrame`: wine 6 or wine 36? (Do not scale the variables.) You should do this for both Euclidian Distance and Manhatten Distance.  Does your answer depend on which distance metric you use to measure "similarity"?

In [ ]:
# YOUR CODE HERE

**Written Answers:**

**Exercise 4.** Now suppose we agree to measure similarity using Euclidean distance, and we wish to investigate the effect of scaling the variables. Which red wine is more similar to wine 0: wine 6 or wine 36? Does the answer depend on whether the variables are scaled or not? Does it depend on the choice of scaling?  What happens for each type of scaling?

In [ ]:
# YOUR CODE HERE

**Written Answers Here:**

### When you have filled out all the questions, submit via [Tulane Canvas](https://tulane.instructure.com/)